In [5]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import matplotlib


pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_columns', 1000000)

## Things to be prepared:
1. month, season, circular month
2. amount spent on previous shopping week
3. amount spent on previous week in time (might be 0)
4. number of weeks since last shopping
5. number of days since last shopping
6. average spent weekly on previous weeks (only weeks existent in data)
    * stats for 6
7. average spent weekly on previous weeks (with actual week count)
    * stats for 7
8. weighted mean of previous week spendings
9. average visits per week
10. average visits per month
11. (LOW PRIORITY) proportion of shopping on weekdays on previous weeks
12. (LOW PRIORITY) proportion of shopping at different times of day on previous weeks
13. demographics

#. clustered demographics

In [19]:
day_week_lookup = df_transactions[['DAY','WEEK_NO']].drop_duplicates().reset_index().drop('index',axis=1)
print(day_week_lookup.shape)
day_week_lookup.head(10)

(711, 2)


,DAY,WEEK_NO
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
5,6,2
6,7,2
7,8,2
8,9,2
9,10,2


In [20]:
week_month_lookup = day_week_lookup.copy()
week_month_lookup['month_num'] = week_month_lookup['DAY'].apply(add_month_nums)
week_month_lookup.drop('DAY', axis=1, inplace=True)
week_month_lookup.drop_duplicates(subset=['WEEK_NO'], keep="last", inplace=True)

print(week_month_lookup.shape)
week_month_lookup.head(10)

(102, 2)


,WEEK_NO,month_num
4,1,2
11,2,3
18,3,3
25,4,3
32,5,3
39,6,4
46,7,4
53,8,4
60,9,4
67,10,4


## 13. demographics

In [55]:
df_demographics = pd.read_csv('dunnhumby-the-complete-journey/hh_demographic.csv')

def house_structure(composition):
    if 'Single' in composition:
        return 1, 0
    elif 'No' in composition:
        return 2, 0
    elif 'No' not in composition and composition != 'Unknown':
        return 2, 1
    else:
        return None, None

df_demographics['Adults'], df_demographics['Child Boolean'] = zip(*df_demographics['HH_COMP_DESC'].apply(house_structure))
                
df_demographics['KID_CATEGORY_DESC'] = df_demographics['KID_CATEGORY_DESC'].apply(lambda x: 3 if x == '3+' else x)
df_demographics['KID_CATEGORY_DESC'] = df_demographics['KID_CATEGORY_DESC'].apply(lambda x: 0 if x == 'None/Unknown' else int(x))

df_demographics['HOUSEHOLD_SIZE_DESC'] = df_demographics['HOUSEHOLD_SIZE_DESC'].apply(lambda x: 5 if x == '5+' else int(x))

df_demographics['HH_COMP_DESC'] = df_demographics['HH_COMP_DESC'].apply(lambda x: None if x == 'Unknown' else x)

df_demographics['HOMEOWNER_DESC'] = df_demographics['HOMEOWNER_DESC'].apply(lambda x: None if x == 'Unknown' else x)
df_demographics.drop('HH_COMP_DESC',axis=1,inplace=True)

df_demographics['Adults'] = df_demographics['Adults'].astype(float)

print(df_demographics.shape)
df_demographics.head()


(801, 9)


,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key,Adults,Child Boolean
0,65+,A,35-49K,Homeowner,2,0,1,2.0,0
1,45-54,A,50-74K,Homeowner,2,0,7,2.0,0
2,25-34,U,25-34K,None,3,1,8,2.0,1
3,25-34,U,75-99K,Homeowner,4,2,13,2.0,1
4,45-54,B,50-74K,Homeowner,1,0,16,1.0,0


In [56]:
df_demographics.dtypes

AGE_DESC                object
MARITAL_STATUS_CODE     object
INCOME_DESC             object
HOMEOWNER_DESC          object
HOUSEHOLD_SIZE_DESC      int64
KID_CATEGORY_DESC        int64
household_key            int64
Adults                 float64
Child Boolean           object
dtype: object

In [ ]:
# training_data = training_data.merge(df_demographics, on=['household_key'], how='left')


# training_data.shape


## 11. Week days shopping proportion

In [ ]:

hh_key_list = family_analysis['household_key'].drop_duplicates().tolist()

big_list = []
for family in hh_key_list:
    new_row = [family]
    family_df = family_analysis[family_analysis['household_key']==family]
    
    percentage_list = daily_shopping_percentages(family_df)
    new_row = new_row + percentage_list
    big_list.append(new_row)
    
daily_visit_percentages = pd.DataFrame(columns = ['household_key', 'Monday %','Tuesday %','Wednesday %',' Thursday %','Friday %','Saturday %','Sunday %','morning %', 'afternoon %', 'evening %'], data=big_list)


In [85]:
shopping_days = df_transactions[['household_key','DAY','WEEK_NO','BASKET_ID']].drop_duplicates()

In [86]:
shopping_days['weekday'] = shopping_days['DAY'].apply(add_week_day_names)

In [87]:
shopping_days = shopping_days.groupby(['household_key','WEEK_NO','weekday'])['BASKET_ID'].count().reset_index()

In [91]:
shopping_days.sort_values(['household_key','WEEK_NO']).head(20)

,household_key,WEEK_NO,weekday,BASKET_ID
0,1,8,Wednesday,1
1,1,10,Friday,1
2,1,13,Friday,1
3,1,14,Thursday,1
4,1,15,Thursday,1
5,1,16,Thursday,1
6,1,17,Sunday,1
7,1,19,Wednesday,1
8,1,20,Friday,1
9,1,22,Sunday,1


In [98]:
family_stats = pd.read_csv('shopping habits of families.csv')
family_stats.drop(['Unnamed: 0','weekly_avg_spending_groceries','weekly_avg_spending_fuel','average_visits_per_week'],axis=1,inplace=True)
print(family_stats.shape)
family_stats.head()


(2500, 11)


,household_key,Monday %,Tuesday %,Wednesday %,Thursday %,Friday %,Saturday %,Sunday %,morning %,afternoon %,evening %
0,1,0.073538,0.187609,0.264042,0.132021,0.220035,0.044007,0.078749,0.285466,0.627678,0.086856
1,2,0.063025,0.029412,0.138655,0.245098,0.120448,0.141457,0.261905,0.078431,0.333333,0.588235
2,3,0.129207,0.216069,0.106406,0.192182,0.071661,0.279045,0.005429,0.013029,0.694897,0.292074
3,4,0.176080,0.136213,0.000000,0.192691,0.023256,0.106312,0.365449,0.116279,0.514950,0.368771
4,5,0.171171,0.040541,0.009009,0.135135,0.063063,0.175676,0.405405,0.000000,0.333333,0.666667


## 10. average visits per month

In [51]:
transactions_with_time = df_transactions[['household_key','WEEK_NO','month_num']].drop_duplicates()
transactions_with_time['year'] = transactions_with_time['WEEK_NO'].apply(lambda x: 2019 if x<45 else 2020 )
transactions_with_time = transactions_with_time.drop('WEEK_NO', axis=1).drop_duplicates().reset_index().drop('index',axis=1)
print(transactions_with_time.shape)
transactions_with_time.head()

(42701, 3)


,household_key,month_num,year
0,2375,2,2019
1,1364,2,2019
2,1130,2,2019
3,1173,2,2019
4,98,2,2019


In [52]:
subset = df_transactions[['household_key','BASKET_ID','WEEK_NO','month_num']].drop_duplicates()
subset['year'] = subset['WEEK_NO'].apply(lambda x: 2019 if x<45 else 2020 )
subset.drop('WEEK_NO',inplace=True,axis=1)
subset = subset.groupby(['household_key','month_num','year']).count().reset_index()

subset.rename(columns={'BASKET_ID':'count_basket_id'}, inplace=True)

print(subset.shape)
subset.head()

(42701, 4)


,household_key,month_num,year,count_basket_id
0,1,1,2020,6
1,1,2,2020,1
2,1,3,2020,9
3,1,4,2019,2
4,1,4,2020,4


In [53]:

family_monthly_visits = subset.sort_values(by=['month_num','year']).groupby('household_key').expanding().mean()

family_monthly_visits = family_monthly_visits.drop('household_key', axis=1).reset_index()
family_monthly_visits.drop('level_1', inplace=True, axis=1)

family_monthly_visits = family_monthly_visits.join(transactions_with_time[['household_key','month_num','year']], lsuffix='_avg')
family_monthly_visits = family_monthly_visits.drop(['household_key_avg','year_avg','month_num_avg'], axis=1).reset_index().drop('index',axis=1)
family_monthly_visits['avg_visits_per_month'] = family_monthly_visits.groupby('household_key')['count_basket_id'].shift(1)
family_monthly_visits = family_monthly_visits.drop('count_basket_id', axis=1)

print(family_monthly_visits.shape)
family_monthly_visits.head(10)


(42701, 4)


,household_key,month_num,year,avg_visits_per_month
0,2375,2,2019,NaN
1,1364,2,2019,NaN
2,1130,2,2019,NaN
3,1173,2,2019,NaN
4,98,2,2019,NaN
5,1172,2,2019,NaN
6,1060,2,2019,NaN
7,1351,2,2019,NaN
8,744,2,2019,NaN
9,212,2,2019,NaN


In [54]:
family_monthly_visits.sort_values(['household_key','year','month_num']).head(200)

,household_key,month_num,year,avg_visits_per_month
1482,1,4,2019,NaN
3300,1,5,2019,2.666667
4524,1,6,2019,2.416667
7133,1,7,2019,1.500000
8952,1,8,2019,8.300000
10670,1,9,2019,3.333333
12706,1,10,2019,2.230769
15468,1,11,2019,5.818182
16493,1,12,2019,1.666667
18998,1,1,2020,4.750000


In [ ]:
# training_data = training_data.merge(family_monthly_visits, on=['household_key','month_num','year'], how='left')


# training_data.shape


## 9. average visits per week

In [48]:
transactions_with_time = transactions_weekly_aggregated.copy()
print(transactions_with_time.shape)
transactions_with_time.head()

(123976, 3)


,household_key,WEEK_NO,SALES_VALUE
0,1,8,78.66
1,1,10,41.10
2,1,13,26.90
3,1,14,63.43
4,1,15,53.45


In [49]:
family_weekly_visits = df_transactions[['household_key','WEEK_NO','BASKET_ID']].drop_duplicates()
family_weekly_visits = family_weekly_visits.groupby(['household_key','WEEK_NO']).count().reset_index()
family_weekly_visits.rename(columns={'BASKET_ID':'count_basket_id'}, inplace=True)
 
family_weekly_visits.head()


,household_key,WEEK_NO,count_basket_id
0,1,8,1
1,1,10,1
2,1,13,1
3,1,14,1
4,1,15,1


In [50]:

family_weekly_visits = family_weekly_visits.sort_values(by='WEEK_NO').groupby('household_key').expanding().mean()
family_weekly_visits = family_weekly_visits.drop('household_key', axis=1).reset_index()
family_weekly_visits.index = family_weekly_visits['level_1']

family_weekly_visits = family_weekly_visits.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
family_weekly_visits = family_weekly_visits.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
family_weekly_visits['avg_visits_per_week'] = family_weekly_visits.groupby('household_key')['count_basket_id'].shift(1)
family_weekly_visits = family_weekly_visits.drop('count_basket_id', axis=1)

print(family_weekly_visits.shape)
family_weekly_visits.head(10)


(123976, 3)


,household_key,WEEK_NO,avg_visits_per_week
0,1,8,NaN
1,1,10,1.0
2,1,13,1.0
3,1,14,1.0
4,1,15,1.0
5,1,16,1.0
6,1,17,1.0
7,1,19,1.0
8,1,20,1.0
9,1,22,1.0


In [ ]:
# training_data = training_data.merge(family_weekly_visits, on=['household_key','WEEK_NO'], how='left')


# training_data.shape


## 8. weighted mean of previous week spendings

In [44]:
transactions_weekly_aggregated.head()

,household_key,WEEK_NO,SALES_VALUE
0,1,8,78.66
1,1,10,41.10
2,1,13,26.90
3,1,14,63.43
4,1,15,53.45


In [45]:
weighted_avg = transactions_weekly_aggregated.copy()
weighted_avg['1_times_before'] = weighted_avg.groupby('household_key')['SALES_VALUE'].shift(1)
weighted_avg['2_times_before'] = weighted_avg.groupby('household_key')['SALES_VALUE'].shift(2)
weighted_avg['3_times_before'] = weighted_avg.groupby('household_key')['SALES_VALUE'].shift(3)
weighted_avg['4_times_before'] = weighted_avg.groupby('household_key')['SALES_VALUE'].shift(4)
weighted_avg['5_times_before'] = weighted_avg.groupby('household_key')['SALES_VALUE'].shift(5)
weighted_avg['6_times_before'] = weighted_avg.groupby('household_key')['SALES_VALUE'].shift(6)
weighted_avg['7_times_before'] = weighted_avg.groupby('household_key')['SALES_VALUE'].shift(7)
weighted_avg['8_times_before'] = weighted_avg.groupby('household_key')['SALES_VALUE'].shift(8)
weighted_avg['9_times_before'] = weighted_avg.groupby('household_key')['SALES_VALUE'].shift(9)
weighted_avg['10_times_before'] = weighted_avg.groupby('household_key')['SALES_VALUE'].shift(10)
weighted_avg.head(20)

,household_key,WEEK_NO,SALES_VALUE,1_times_before,2_times_before,3_times_before,4_times_before,5_times_before,6_times_before,7_times_before,8_times_before,9_times_before,10_times_before
0,1,8,78.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10,41.10,78.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,13,26.90,41.10,78.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,14,63.43,26.90,41.10,78.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,15,53.45,63.43,26.90,41.10,78.66,NaN,NaN,NaN,NaN,NaN,NaN
5,1,16,26.76,53.45,63.43,26.90,41.10,78.66,NaN,NaN,NaN,NaN,NaN
6,1,17,23.55,26.76,53.45,63.43,26.90,41.10,78.66,NaN,NaN,NaN,NaN
7,1,19,110.34,23.55,26.76,53.45,63.43,26.90,41.10,78.66,NaN,NaN,NaN
8,1,20,87.44,110.34,23.55,26.76,53.45,63.43,26.90,41.10,78.66,NaN,NaN
9,1,22,73.32,87.44,110.34,23.55,26.76,53.45,63.43,26.90,41.10,78.66,NaN


In [46]:
weighted_avg['weighted_avg'] = (weighted_avg['1_times_before']*10 + 
                               weighted_avg['2_times_before']*9 +
                               weighted_avg['3_times_before']*8 +
                               weighted_avg['4_times_before']*7 +
                               weighted_avg['5_times_before']*6 +
                               weighted_avg['6_times_before']*5 +
                               weighted_avg['7_times_before']*4 +
                               weighted_avg['8_times_before']*3 +
                               weighted_avg['9_times_before']*2 +
                               weighted_avg['9_times_before']*1)/(1+2+3+4+5+6+7+8+9+10)

In [47]:
weighted_avg = weighted_avg[['household_key','WEEK_NO','weighted_avg']]
weighted_avg.head(200)

,household_key,WEEK_NO,weighted_avg
0,1,8,NaN
1,1,10,NaN
2,1,13,NaN
3,1,14,NaN
4,1,15,NaN
5,1,16,NaN
6,1,17,NaN
7,1,19,NaN
8,1,20,NaN
9,1,22,60.871091


In [ ]:
# training_data = training_data.merge(weighted_avg, on=['household_key','WEEK_NO'], how='left')


# training_data.shape


## 7. average spent weekly on previous weeks (with actual week count)

In [30]:
print(big_df.shape)
big_df.head()

(255000, 3)


,WEEK_NO,household_key,SALES_VALUE
0,1,1,0.0
1,2,1,0.0
2,3,1,0.0
3,4,1,0.0
4,5,1,0.0


In [31]:
mean = big_df[['household_key','WEEK_NO','SALES_VALUE']]
mean = mean.sort_values(by='WEEK_NO').groupby('household_key').expanding().mean()
mean = mean.drop('household_key', axis=1).reset_index()
mean.index = mean['level_1']

mean = mean.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
mean = mean.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
mean['avg_spending_previous_weeks_real_time'] = mean.groupby('household_key')['SALES_VALUE'].shift(1)
mean_r = mean.drop('SALES_VALUE', axis=1)
print(mean_r.shape)
mean_r.head(10)


(255000, 3)


,household_key,WEEK_NO,avg_spending_previous_weeks_real_time
0,1.0,8.0,NaN
1,1.0,10.0,0.0000
2,1.0,13.0,0.0000
3,1.0,14.0,0.0000
4,1.0,15.0,0.0000
5,1.0,16.0,0.0000
6,1.0,17.0,0.0000
7,1.0,19.0,0.0000
8,1.0,20.0,9.8325
9,1.0,22.0,8.7400


In [32]:
std = big_df[['household_key','WEEK_NO','SALES_VALUE']]
std = std.sort_values(by='WEEK_NO').groupby('household_key').expanding().std()
std = std.drop('household_key', axis=1).reset_index()
std.index = std['level_1']

std = std.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
std = std.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
std['std_spending_previous_weeks_real_time'] = std.groupby('household_key')['SALES_VALUE'].shift(1)
std_r = std.drop('SALES_VALUE', axis=1)
print(std_r.shape)
std_r.head()



(255000, 3)


,household_key,WEEK_NO,std_spending_previous_weeks_real_time
0,1.0,8.0,NaN
1,1.0,10.0,NaN
2,1.0,13.0,0.0
3,1.0,14.0,0.0
4,1.0,15.0,0.0


In [33]:
var = big_df[['household_key','WEEK_NO','SALES_VALUE']]
var = var.sort_values(by='WEEK_NO').groupby('household_key').expanding().var()
var = var.drop('household_key', axis=1).reset_index()
var.index = var['level_1']

var = var.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
var = var.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
var['var_spending_previous_weeks_real_time'] = var.groupby('household_key')['SALES_VALUE'].shift(1)
var_r = var.drop('SALES_VALUE', axis=1)
print(var_r.shape)
var_r.head(10)


(255000, 3)


,household_key,WEEK_NO,var_spending_previous_weeks_real_time
0,1.0,8.0,NaN
1,1.0,10.0,NaN
2,1.0,13.0,0.00000
3,1.0,14.0,0.00000
4,1.0,15.0,0.00000
5,1.0,16.0,0.00000
6,1.0,17.0,0.00000
7,1.0,19.0,0.00000
8,1.0,20.0,773.42445
9,1.0,22.0,687.48840


In [34]:
quan25 = big_df[['household_key','WEEK_NO','SALES_VALUE']]
quan25 = quan25.sort_values(by='WEEK_NO').groupby('household_key').expanding().quantile(0.25)
quan25 = quan25.drop('household_key', axis=1).reset_index()
quan25.index = quan25['level_1']

quan25 = quan25.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
quan25 = quan25.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
quan25['quan25_spending_previous_weeks_real_time'] = quan25.groupby('household_key')['SALES_VALUE'].shift(1)
quan25_r = quan25.drop('SALES_VALUE', axis=1)
print(quan25_r.shape)
quan25_r.head(10)


(255000, 3)


,household_key,WEEK_NO,quan25_spending_previous_weeks_real_time
0,1.0,8.0,NaN
1,1.0,10.0,0.0
2,1.0,13.0,0.0
3,1.0,14.0,0.0
4,1.0,15.0,0.0
5,1.0,16.0,0.0
6,1.0,17.0,0.0
7,1.0,19.0,0.0
8,1.0,20.0,0.0
9,1.0,22.0,0.0


In [35]:
quan50 = big_df[['household_key','WEEK_NO','SALES_VALUE']]
quan50 = quan50.sort_values(by='WEEK_NO').groupby('household_key').expanding().quantile(0.5)
quan50 = quan50.drop('household_key', axis=1).reset_index()
quan50.index = quan50['level_1']

quan50 = quan50.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
quan50 = quan50.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
quan50['quan50_spending_previous_weeks_real_time'] = quan50.groupby('household_key')['SALES_VALUE'].shift(1)
quan50_r = quan50.drop('SALES_VALUE', axis=1)
print(quan50_r.shape)
quan50_r.head(10)


(255000, 3)


,household_key,WEEK_NO,quan50_spending_previous_weeks_real_time
0,1.0,8.0,NaN
1,1.0,10.0,0.0
2,1.0,13.0,0.0
3,1.0,14.0,0.0
4,1.0,15.0,0.0
5,1.0,16.0,0.0
6,1.0,17.0,0.0
7,1.0,19.0,0.0
8,1.0,20.0,0.0
9,1.0,22.0,0.0


In [36]:
quan75 = big_df[['household_key','WEEK_NO','SALES_VALUE']]
quan75 = quan75.sort_values(by='WEEK_NO').groupby('household_key').expanding().quantile(0.75)
quan75 = quan75.drop('household_key', axis=1).reset_index()
quan75.index = quan75['level_1']

quan75 = quan75.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
quan75 = quan75.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
quan75['quan75_spending_previous_weeks_real_time'] = quan75.groupby('household_key')['SALES_VALUE'].shift(1)
quan75_r = quan75.drop('SALES_VALUE', axis=1)
print(quan75_r.shape)
quan75_r.head(10)


(255000, 3)


,household_key,WEEK_NO,quan75_spending_previous_weeks_real_time
0,1.0,8.0,NaN
1,1.0,10.0,0.0
2,1.0,13.0,0.0
3,1.0,14.0,0.0
4,1.0,15.0,0.0
5,1.0,16.0,0.0
6,1.0,17.0,0.0
7,1.0,19.0,0.0
8,1.0,20.0,0.0
9,1.0,22.0,0.0


In [ ]:
# training_data = training_data.merge(mean, on=['household_key','WEEK_NO'], how='left')
# training_data = training_data.merge(std, on=['household_key','WEEK_NO'], how='left')
# training_data = training_data.merge(var, on=['household_key','WEEK_NO'], how='left')
# training_data = training_data.merge(quan25, on=['household_key','WEEK_NO'], how='left')
# training_data = training_data.merge(quan50, on=['household_key','WEEK_NO'], how='left')
# training_data = training_data.merge(quan75, on=['household_key','WEEK_NO'], how='left')


# training_data.shape


## 6. average spent weekly on previous weeks (only weeks existent in data)

In [37]:
transactions_with_time = transactions_weekly_aggregated.copy()
print(transactions_with_time.shape)
transactions_with_time.head()



(123976, 3)


,household_key,WEEK_NO,SALES_VALUE
0,1,8,78.66
1,1,10,41.10
2,1,13,26.90
3,1,14,63.43
4,1,15,53.45


In [38]:
mean = transactions_with_time[['household_key','WEEK_NO','SALES_VALUE']]
mean = mean.sort_values(by='WEEK_NO').groupby('household_key').expanding().mean()
mean = mean.drop('household_key', axis=1).reset_index()
mean.index = mean['level_1']

mean = mean.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
mean = mean.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
mean['avg_spending_previous_weeks'] = mean.groupby('household_key')['SALES_VALUE'].shift(1)
mean = mean.drop('SALES_VALUE', axis=1)
print(mean.shape)
mean.head(100)


(123976, 3)


,household_key,WEEK_NO,avg_spending_previous_weeks
0,1,8,NaN
1,1,10,78.660000
2,1,13,59.880000
3,1,14,48.886667
4,1,15,52.522500
5,1,16,52.708000
6,1,17,48.383333
7,1,19,44.835714
8,1,20,53.023750
9,1,22,56.847778


In [39]:
std = transactions_with_time[['household_key','WEEK_NO','SALES_VALUE']]
std = std.sort_values(by='WEEK_NO').groupby('household_key').expanding().std()
std = std.drop('household_key', axis=1).reset_index()
std.index = std['level_1']

std = std.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
std = std.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
std['std_spending_previous_weeks'] = std.groupby('household_key')['SALES_VALUE'].shift(1)
std = std.drop('SALES_VALUE', axis=1)
print(std.shape)
std.head(100)



(123976, 3)


,household_key,WEEK_NO,std_spending_previous_weeks
0,1,8,NaN
1,1,10,NaN
2,1,13,26.558931
3,1,14,26.744131
4,1,15,23.015419
5,1,16,19.936253
6,1,17,20.740776
7,1,19,21.132495
8,1,20,30.317261
9,1,22,30.591714


In [40]:
var = transactions_with_time[['household_key','WEEK_NO','SALES_VALUE']]
var = var.sort_values(by='WEEK_NO').groupby('household_key').expanding().var()
var = var.drop('household_key', axis=1).reset_index()
var.index = var['level_1']

var = var.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
var = var.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
var['var_spending_previous_weeks'] = var.groupby('household_key')['SALES_VALUE'].shift(1)
var = var.drop('SALES_VALUE', axis=1)
print(var.shape)
var.head(100)



(123976, 3)


,household_key,WEEK_NO,var_spending_previous_weeks
0,1,8,NaN
1,1,10,NaN
2,1,13,705.376800
3,1,14,715.248533
4,1,15,529.709492
5,1,16,397.454170
6,1,17,430.179787
7,1,19,446.582362
8,1,20,919.136312
9,1,22,935.852969


In [41]:
quan25 = transactions_with_time[['household_key','WEEK_NO','SALES_VALUE']]
quan25 = quan25.sort_values(by='WEEK_NO').groupby('household_key').expanding().quantile(0.25)
quan25 = quan25.drop('household_key', axis=1).reset_index()
quan25.index = quan25['level_1']

quan25 = quan25.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
quan25 = quan25.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
quan25['quan25_spending_previous_weeks'] = quan25.groupby('household_key')['SALES_VALUE'].shift(1)
quan25 = quan25.drop('SALES_VALUE', axis=1)
print(quan25.shape)
quan25.head(100)


(123976, 3)


,household_key,WEEK_NO,quan25_spending_previous_weeks
0,1,8,NaN
1,1,10,78.6600
2,1,13,50.4900
3,1,14,34.0000
4,1,15,37.5500
5,1,16,41.1000
6,1,17,30.4500
7,1,19,26.8300
8,1,20,26.8650
9,1,22,26.9000


In [42]:
quan50 = transactions_with_time[['household_key','WEEK_NO','SALES_VALUE']]
quan50 = quan50.sort_values(by='WEEK_NO').groupby('household_key').expanding().quantile(0.5)
quan50 = quan50.drop('household_key', axis=1).reset_index()
quan50.index = quan50['level_1']

quan50 = quan50.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
quan50 = quan50.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
quan50['quan50_spending_previous_weeks'] = quan50.groupby('household_key')['SALES_VALUE'].shift(1)
quan50 = quan50.drop('SALES_VALUE', axis=1)
print(quan50.shape)
quan50.head(100)



(123976, 3)


,household_key,WEEK_NO,quan50_spending_previous_weeks
0,1,8,NaN
1,1,10,78.660
2,1,13,59.880
3,1,14,41.100
4,1,15,52.265
5,1,16,53.450
6,1,17,47.275
7,1,19,41.100
8,1,20,47.275
9,1,22,53.450


In [43]:
quan75 = transactions_with_time[['household_key','WEEK_NO','SALES_VALUE']]
quan75 = quan75.sort_values(by='WEEK_NO').groupby('household_key').expanding().quantile(0.75)
quan75 = quan75.drop('household_key', axis=1).reset_index()
quan75.index = quan75['level_1']

quan75 = quan75.join(transactions_with_time[['household_key','WEEK_NO']], lsuffix='_avg')
quan75 = quan75.drop(['household_key_avg','level_1','WEEK_NO_avg'], axis=1).reset_index().drop('level_1', axis=1)
quan75['quan75_spending_previous_weeks'] = quan75.groupby('household_key')['SALES_VALUE'].shift(1)
quan75 = quan75.drop('SALES_VALUE', axis=1)
print(quan75.shape)
quan75.head(100)



(123976, 3)


,household_key,WEEK_NO,quan75_spending_previous_weeks
0,1,8,NaN
1,1,10,78.6600
2,1,13,69.2700
3,1,14,59.8800
4,1,15,67.2375
5,1,16,63.4300
6,1,17,60.9350
7,1,19,58.4400
8,1,20,67.2375
9,1,22,78.6600


In [ ]:
# training_data = training_data.merge(mean, on=['household_key','WEEK_NO'], how='left')
# training_data = training_data.merge(std, on=['household_key','WEEK_NO'], how='left')
# training_data = training_data.merge(var, on=['household_key','WEEK_NO'], how='left')
# training_data = training_data.merge(quan25, on=['household_key','WEEK_NO'], how='left')
# training_data = training_data.merge(quan50, on=['household_key','WEEK_NO'], how='left')
# training_data = training_data.merge(quan75, on=['household_key','WEEK_NO'], how='left')

# training_data.shape


## 5. number of days since last shopping

In [21]:
previous_visit_day = df_transactions[['household_key','DAY']].drop_duplicates().sort_values(['household_key','DAY'])
previous_visit_day['last_shopping_day'] = previous_visit_day.groupby('household_key')['DAY'].shift(1)
previous_visit_day['days_since_last_shopping'] = previous_visit_day['DAY']-previous_visit_day['last_shopping_day']
previous_visit_day = previous_visit_day.merge(day_week_lookup, on='DAY', how='left')

print(previous_visit_day.shape)
previous_visit_day.head(10)


(225533, 5)


,household_key,DAY,last_shopping_day,days_since_last_shopping,WEEK_NO
0,1,51,NaN,NaN,8
1,1,67,51.0,16.0,10
2,1,88,67.0,21.0,13
3,1,94,88.0,6.0,14
4,1,101,94.0,7.0,15
5,1,108,101.0,7.0,16
6,1,111,108.0,3.0,17
7,1,128,111.0,17.0,19
8,1,137,128.0,9.0,20
9,1,146,137.0,9.0,22


In [22]:
# this data is indexed on DAY but our overall dataset is on week level
# So I get 1 value per week (the higher one)

previous_visit_day = previous_visit_day.sort_values('days_since_last_shopping')
previous_visit_day = previous_visit_day.drop_duplicates(subset=['household_key','WEEK_NO'], keep="last")
print(previous_visit_day.shape)
previous_visit_day.head(10)

(123976, 5)


,household_key,DAY,last_shopping_day,days_since_last_shopping,WEEK_NO
99034,1079,391,390.0,1.0,57
198341,2198,160,159.0,1.0,24
198376,2198,244,243.0,1.0,36
167514,1850,322,321.0,1.0,47
198415,2198,384,383.0,1.0,56
45129,510,629,628.0,1.0,91
45092,510,545,544.0,1.0,79
167158,1847,426,425.0,1.0,62
44161,493,342,341.0,1.0,50
99658,1086,244,243.0,1.0,36


In [ ]:
# training_data = training_data.merge(previous_visit_day, on=['household_key','WEEK_NO'], how='left')

# training_data.shape


## 4. number of weeks since last shopping

In [15]:
transactions_prev_visit = transactions_weekly_aggregated.copy()
transactions_prev_visit['last_shopping_week'] = transactions_prev_visit.groupby('household_key')['WEEK_NO'].shift(1)
transactions_prev_visit['weeks_since_last_shopping'] = transactions_prev_visit['WEEK_NO']-transactions_prev_visit['last_shopping_week']

transactions_prev_visit = transactions_prev_visit.drop('SALES_VALUE', axis=1)


In [16]:
transactions_prev_visit.head()

,household_key,WEEK_NO,last_shopping_week,weeks_since_last_shopping
0,1,8,NaN,NaN
1,1,10,8.0,2.0
2,1,13,10.0,3.0
3,1,14,13.0,1.0
4,1,15,14.0,1.0


In [ ]:
# training_data = training_data.merge(transactions_prev_visit, on=['household_key','WEEK_NO'], how='left')

# training_data.shape


## 3. amount spent on previous week in time (might be 0)

In [12]:
households = df_transactions['household_key'].sort_values().unique()

big_df = pd.DataFrame()

for key in households:
    df = pd.DataFrame()
    df['WEEK_NO'] = np.arange(1,103)
    df['household_key'] = key
    
    big_df = pd.concat([big_df,df])

print(big_df.shape)
big_df = big_df.merge(transactions_weekly_aggregated, on=['household_key','WEEK_NO'], how='left')
print(big_df.shape)

big_df = big_df.fillna(0)
big_df = big_df.sort_values(by=['household_key','WEEK_NO']).reset_index().drop('index', axis=1)
big_df.head(10)

(255000, 2)
(255000, 3)


,WEEK_NO,household_key,SALES_VALUE
0,1,1,0.00
1,2,1,0.00
2,3,1,0.00
3,4,1,0.00
4,5,1,0.00
5,6,1,0.00
6,7,1,0.00
7,8,1,78.66
8,9,1,0.00
9,10,1,41.10


In [13]:
big_df_aggregated = big_df.copy()
big_df_aggregated['amount_spent_last_week'] = big_df_aggregated.sort_values('WEEK_NO').groupby('household_key')['SALES_VALUE'].shift(1)
big_df_aggregated = big_df_aggregated.drop('SALES_VALUE', axis=1)
print(big_df_aggregated.shape)
big_df_aggregated.head()


(255000, 3)


,WEEK_NO,household_key,amount_spent_last_week
0,1,1,NaN
1,2,1,0.0
2,3,1,0.0
3,4,1,0.0
4,5,1,0.0


In [14]:
# training_data = training_data.merge(big_df_aggregated, on=['household_key','WEEK_NO'], how='left')

# training_data.shape


## 2. amount spent on previous shopping week

In [9]:
# Summing individual transactions to have total basket value
transactions_weekly_aggregated = df_transactions.groupby(["household_key", "WEEK_NO"])["SALES_VALUE"].sum().reset_index()
transactions_weekly_aggregated = transactions_weekly_aggregated.sort_values(by=['household_key','WEEK_NO']).reset_index().drop('index', axis=1)

print(transactions_weekly_aggregated.shape)
transactions_weekly_aggregated.head(10)


(123976, 3)


,household_key,WEEK_NO,SALES_VALUE
0,1,8,78.66
1,1,10,41.10
2,1,13,26.90
3,1,14,63.43
4,1,15,53.45
5,1,16,26.76
6,1,17,23.55
7,1,19,110.34
8,1,20,87.44
9,1,22,73.32


In [10]:
transactions_weekly_aggregated_prev = transactions_weekly_aggregated.copy()
transactions_weekly_aggregated_prev['previous_amount_spent'] = transactions_weekly_aggregated_prev.sort_values('WEEK_NO').groupby('household_key')['SALES_VALUE'].shift(1)
transactions_weekly_aggregated_prev = transactions_weekly_aggregated_prev.drop('SALES_VALUE', axis=1)
print(transactions_weekly_aggregated_prev.shape)
transactions_weekly_aggregated_prev.head()


(123976, 3)


,household_key,WEEK_NO,previous_amount_spent
0,1,8,NaN
1,1,10,78.66
2,1,13,41.10
3,1,14,26.90
4,1,15,63.43


In [11]:
# training_data = training_data.merge(transactions_weekly_aggregated_prev, on=['household_key','WEEK_NO'], how='left')



## 1. month, season, circular month

In [6]:
df_transactions = pd.read_csv('dunnhumby-the-complete-journey/transaction_data.csv')
print('number of all transactions: ' +str(df_transactions.shape[0]))
df_transactions.head()

number of all transactions: 2595732


,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [7]:
df_transactions['month'] = df_transactions['DAY'].apply(add_month_names)
df_transactions['month_num'] = df_transactions['DAY'].apply(add_month_nums)

df_transactions['season'] = df_transactions['month'].apply(get_season)



In [8]:
month_season_lookup_table = df_transactions[['WEEK_NO','month','month_num','season']].drop_duplicates().reset_index().drop('index', axis=1)
month_season_lookup_table = month_season_lookup_table.drop_duplicates(subset='WEEK_NO', keep="first").reset_index().drop('index', axis=1)

# dealing with circular features
month_season_lookup_table['month_24']= month_season_lookup_table['month_num']*2
month_season_lookup_table['month_x'], month_season_lookup_table['month_y']= zip(*month_season_lookup_table['month_24'].apply(getxy))


month_season_lookup_table.drop(['month_24','month_num'], axis=1, inplace=True)


print(month_season_lookup_table.shape)
month_season_lookup_table


(102, 5)


,WEEK_NO,month,season,month_x,month_y
0,1,February,winter,0.866223,-0.499658
1,2,February,winter,0.866223,-0.499658
2,3,March,spring,1.000000,0.000296
3,4,March,spring,1.000000,0.000296
4,5,March,spring,1.000000,0.000296
5,6,March,spring,1.000000,0.000296
6,7,April,spring,0.865927,0.500171
7,8,April,spring,0.865927,0.500171
8,9,April,spring,0.865927,0.500171
9,10,April,spring,0.865927,0.500171


In [ ]:
# training_data = training_data.merge(month_season_lookup_table, on='WEEK_NO', how='left')

In [ ]:
print(training_data.shape)
training_data.head(100)

In [63]:
def add_month_names(day):
    day_number = day%340
    if day_number<10:
        return 'February'
    elif day_number<40:
        return 'March'
    elif day_number<70:
        return 'April'
    elif day_number<100:
        return 'May'
    elif day_number<130:
        return 'June'
    elif day_number<160:
        return 'July'
    elif day_number<190:
        return 'August'
    elif day_number<220:
        return 'September'
    elif day_number<250:
        return 'October'
    elif day_number<280:
        return 'November'
    elif day_number<310:
        return 'December'
    elif day_number<340:
        return 'January'
    
def add_month_nums(day):
    day_number = day%340
    if day_number<10:
        return 2
    elif day_number<40:
        return 3
    elif day_number<70:
        return 4
    elif day_number<100:
        return 5
    elif day_number<130:
        return 6
    elif day_number<160:
        return 7
    elif day_number<190:
        return 8
    elif day_number<220:
        return 9
    elif day_number<250:
        return 10
    elif day_number<280:
        return 11
    elif day_number<310:
        return 12
    elif day_number<340:
        return 1
    
def add_week_day_names(day):
    day_number = day%7
    if day_number==1:
        return 'Tuesday'
    elif day_number==2:
        return 'Wednesday'
    elif day_number==3:
        return 'Thursday'
    elif day_number==4:
        return 'Friday'
    elif day_number==5:
        return 'Saturday'
    elif day_number==6:
        return 'Sunday'
    elif day_number==0:
        return 'Monday'
    
def add_week_day_nums(day):
    day_number = day%7
    if day_number==1:
        return 2
    elif day_number==2:
        return 3
    elif day_number==3:
        return 4
    elif day_number==4:
        return 5
    elif day_number==5:
        return 6
    elif day_number==6:
        return 7
    elif day_number==0:
        return 8
    
    
def get_season(month):
    if month == 'December' or month == 'January' or month == 'February':
        return 'winter'
    elif month == 'March' or month == 'April' or month == 'May':
        return 'spring'
    elif month == 'June' or month == 'July' or month == 'August':
        return 'summer'
    elif month == 'September' or month == 'October' or month == 'November':
        return 'fall'
    
    
def getxy(hour):
    x = math.sin((180 - hour * 15)/180 * 3.141)
    y = math.cos((180 - hour * 15)/180 * 3.141)
    return x, y

In [94]:
training_data = df_transactions[['household_key','WEEK_NO']].drop_duplicates()
training_data = transactions_weekly_aggregated.copy()
training_data = training_data.merge(week_month_lookup, on='WEEK_NO')
training_data['year'] = training_data['WEEK_NO'].apply(lambda x: 2019 if x<45 else 2020 )
training_data.shape

(123976, 5)

In [95]:
training_data.head()

,household_key,WEEK_NO,SALES_VALUE,month_num,year
0,1,8,78.66,4,2019
1,14,8,14.92,4,2019
2,17,8,64.41,4,2019
3,22,8,33.20,4,2019
4,25,8,111.06,4,2019


In [96]:


#13. demographics
training_data = training_data.merge(df_demographics, on=['household_key'], how='left')

training_data = training_data.merge(family_stats, on='household_key', how='left')

#10. average visits per month
training_data = training_data.merge(family_monthly_visits, on=['household_key','month_num','year'], how='left')

#9. average visits per week
training_data = training_data.merge(family_weekly_visits, on=['household_key','WEEK_NO'], how='left')

#8. weighted mean of previous week spendings
training_data = training_data.merge(weighted_avg, on=['household_key','WEEK_NO'], how='left')

#7. average spent weekly on previous weeks (with actual week count)
training_data = training_data.merge(mean_r, on=['household_key','WEEK_NO'], how='left')
training_data = training_data.merge(std_r, on=['household_key','WEEK_NO'], how='left')
training_data = training_data.merge(var_r, on=['household_key','WEEK_NO'], how='left')
training_data = training_data.merge(quan25_r, on=['household_key','WEEK_NO'], how='left')
training_data = training_data.merge(quan50_r, on=['household_key','WEEK_NO'], how='left')
training_data = training_data.merge(quan75_r, on=['household_key','WEEK_NO'], how='left')

#6. average spent weekly on previous weeks (only weeks existent in data)
training_data = training_data.merge(mean, on=['household_key','WEEK_NO'], how='left')
training_data = training_data.merge(std, on=['household_key','WEEK_NO'], how='left')
training_data = training_data.merge(var, on=['household_key','WEEK_NO'], how='left')
training_data = training_data.merge(quan25, on=['household_key','WEEK_NO'], how='left')
training_data = training_data.merge(quan50, on=['household_key','WEEK_NO'], how='left')
training_data = training_data.merge(quan75, on=['household_key','WEEK_NO'], how='left')

#5. number of days since last shopping
training_data = training_data.merge(previous_visit_day, on=['household_key','WEEK_NO'], how='left')

#4. number of weeks since last shopping
training_data = training_data.merge(transactions_prev_visit, on=['household_key','WEEK_NO'], how='left')

#3. amount spent on previous week in time (might be 0)
training_data = training_data.merge(big_df_aggregated, on=['household_key','WEEK_NO'], how='left')

#2. amount spent on previous shopping week
training_data = training_data.merge(transactions_weekly_aggregated_prev, on=['household_key','WEEK_NO'], how='left')

#1. month, season, circular month
training_data = training_data.merge(month_season_lookup_table, on='WEEK_NO', how='left')

training_data.shape



(123976, 49)

In [99]:
training_data.head()

,household_key,WEEK_NO,SALES_VALUE,month_num,year,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,Adults,Child Boolean,Monday %,Tuesday %,Wednesday %,Thursday %,Friday %,Saturday %,Sunday %,morning %,afternoon %,evening %,avg_visits_per_month,avg_visits_per_week,weighted_avg,avg_spending_previous_weeks_real_time,std_spending_previous_weeks_real_time,var_spending_previous_weeks_real_time,quan25_spending_previous_weeks_real_time,quan50_spending_previous_weeks_real_time,quan75_spending_previous_weeks_real_time,avg_spending_previous_weeks,std_spending_previous_weeks,var_spending_previous_weeks,quan25_spending_previous_weeks,quan50_spending_previous_weeks,quan75_spending_previous_weeks,DAY,last_shopping_day,days_since_last_shopping,last_shopping_week,weeks_since_last_shopping,amount_spent_last_week,previous_amount_spent,month,season,month_x,month_y
0,1,8,78.66,4,2019,65+,A,35-49K,Homeowner,2.0,0.0,2.0,0,0.073538,0.187609,0.264042,0.132021,0.220035,0.044007,0.078749,0.285466,0.627678,0.086856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51,NaN,NaN,NaN,NaN,0.00,NaN,April,spring,0.865927,0.500171
1,14,8,14.92,4,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.086420,0.060606,0.109989,0.076319,0.355780,0.156004,0.154882,0.041526,0.159371,0.799102,7.833333,1.0,NaN,0.000000,0.000000,0.000000,0.0,0.0,0.000,65.360000,60.333961,3640.186900,36.0600,68.690,96.3250,53,30.0,23.0,5.0,3.0,0.00,3.43,April,spring,0.865927,0.500171
2,17,8,64.41,4,2019,65+,B,Under 15K,Homeowner,2.0,0.0,2.0,0,0.112705,0.145492,0.163934,0.181352,0.204918,0.000000,0.191598,0.282787,0.658811,0.058402,3.450000,1.5,NaN,26.104118,41.895223,1755.209723,0.0,0.0,42.495,120.140000,54.778341,3000.666600,93.8000,133.565,159.9050,49,44.0,5.0,7.0,1.0,157.76,157.76,April,spring,0.865927,0.500171
3,22,8,33.20,4,2019,45-54,A,75-99K,Homeowner,2.0,0.0,2.0,0,0.076398,0.011801,0.059627,0.095031,0.208075,0.481366,0.067702,0.057764,0.865839,0.076398,4.000000,1.0,NaN,2.015208,10.594933,112.252602,0.0,0.0,0.000,55.780000,25.124470,631.239000,50.6025,60.755,65.9325,49,44.0,5.0,7.0,1.0,20.85,20.85,April,spring,0.865927,0.500171
4,25,8,111.06,4,2019,35-44,U,50-74K,None,1.0,0.0,NaN,None,0.069882,0.037402,0.043307,0.284449,0.388780,0.125984,0.050197,0.057087,0.443898,0.499016,3.235294,1.0,NaN,3.502763,13.769607,189.602079,0.0,0.0,0.000,27.273333,26.457635,700.006433,12.3100,17.630,37.4150,50,47.0,3.0,7.0,1.0,6.99,6.99,April,spring,0.865927,0.500171


In [101]:
# training_data.to_csv('data_with_features.csv')